# CSMCC16 Flight Coursework

### Import packages

In [1]:
import numpy as np
import pandas as pd
import datetime
import re
from difflib import SequenceMatcher
import PySimpleGUI as sg

### Import data

In [8]:
#import csv as dataframe
pass_data = pd.DataFrame(pd.read_csv('AComp_Passenger_data.csv',header=None))
airport_lat_lon = pd.DataFrame(pd.read_csv('Top30_airports_LatLong.csv',header=None))

In [9]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']

### Remove missing values

In [10]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

### Functions for Additional Columns and formats

In [11]:
#calculate datetime values from epoch and arrival time
def calc_arrive_time(df):
    df['Depart_time_datetime'] = pd.to_datetime(df['Depart_time_epoch'],unit='s')
    df['Arrive_epoch'] = pass_data['Depart_time_epoch']+df['Flight_time_min']*60
    df['Arrive_time_datetime'] = pd.to_datetime(df['Arrive_epoch'],unit='s')

In [12]:
#create route (to use as unique id)
def route(df):
    df['Route'] = df['From_airport_IATA']+'-' + df['Dest_airport_IATA']+'-' + df['Depart_time']

In [13]:
#break route for details
def break_route(df):
    df[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=df['Route'].str.split('-',expand=True)

In [ ]:
#break_route(pass_data_route)
#pass_data_route[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=pass_data_route['Route'].str.split('-',expand=True)

In [14]:
#Add new datetime formats
def add_times(df):
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [15]:
#Add column to allow counting of passengers
def add_1(df):
    df['count']='1'

## Setting up the gui output

In [16]:
sg.SetOptions(
    background_color='#152238',
       text_element_background_color='#152238',
       element_background_color='#152238',
       text_color='#FFFFFF',
       scrollbar_color=None,
       input_elements_background_color='#FFFFFF',
       progress_meter_color = ('green', 'blue')
#        button_color=('white','#475841')
    )

True

### Add Additional Columns and formats

In [ ]:
# route(flight_only_data)
####check if necessary!!
add_1(pass_data)
# pass_data
route(pass_data)
# Select Route info only, and group data
key_route_info=['Route'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Flight_time_min'
                      ,'Arrive_time']
route_info=pass_data[key_route_info]
route_info=route_info.groupby(key_route_info).count()
route_info.reset_index(inplace=True)

# def group(base_df,variables,new_df):
#     new_df=pd.DataFrame()
#     #select variables only
#     new_df=base_df[variables]
#     #group by variables to create a table
#     new_df=new_df.groupby(variables).count()
#     #reset index
#     new_df.reset_index(inplace=True)
    
# key_route_info=['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time']
# route_info_new=pd.DataFrame()
# group(pass_data,key_route_info,route_info_new)

# route_info

# Number of passengers per flight id


## Error Correction
### Flights

In [21]:
# #select subset of data 
# flight_info = ['Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_time'         
#                         ]
# flight_only_data = pass_data[flight_info]
# #Group data by flight
# flight_only_data = flight_only_data.groupby(flight_info).count()
# #reset index
# flight_only_data.reset_index(inplace=True)


### Flight data only

In [20]:
#Add flight time info 
calc_arrive_time(pass_data)
add_times(pass_data)

In [25]:
#select subset of data 
flight_info = ['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'         
                        ]
flights = pass_data[flight_info]
#Group data by flight
flights = flights.groupby(flight_info).count()
#reset index
flights.reset_index(inplace=True)
#add route info
route(flights)
flights

In [27]:
#check if airport in airport list
airports = airport_lat_lon['Airport_IATA_code']
airports

In [52]:
# for a in flight_info['From_airport_IATA']:

In [49]:
#Not necessary but keeping for reference

#correct airport format
format_IATA = r"[A-Z][A-Z][A-Z]"
#remove row if from airport not correct format
flights=flights[flights['From_airport_IATA'].str.contains(format_IATA)]
#remove row if dest airport not correct format
flights=flights[flights['Dest_airport_IATA'].str.contains(format_IATA)]

In [50]:
#remove row if flight id not correct format
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"
flights=flights[flights['Flight_id'].str.contains(format_flight_id)]

In [54]:
# check if airport in airport list
flights=flights[flights['From_airport_IATA'].isin(airports)]
flights=flights[flights['Dest_airport_IATA'].isin(airports)]
# flights

In [ ]:
#check data
# flight_only_data

In [55]:
# Calculate the number of passengers per flight
flight_pass = ['Flight_id','Passenger_id']
flight_pass_data = pass_data[flight_pass]
pass_per_flight=flight_pass_data.groupby('Flight_id').count()
pass_per_flight.reset_index(inplace=True)
#pass_per_flight

In [68]:
#merge correct flights with passenger numbers
flights_merge=flights.merge(pass_per_flight, how='inner', on=['Flight_id'])

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time,Route,Passenger_id
0,ATT7791R,AMS,DEN,17:13:14,AMS-DEN-17:13:14,15
1,BER7172M,KUL,LAS,17:26:07,KUL-LAS-17:26:07,17
2,DAU2617A,CGK,SFO,17:23:06,CGK-SFO-17:23:06,12
3,DKZ3042O,MIA,SFO,17:05:27,MIA-SFO-17:05:27,11
4,EWH6301Y,CAN,DFW,17:22:47,CAN-DFW-17:22:47,10
5,FYL5866L,ATL,HKG,17:25:40,ATL-HKG-17:25:40,20
6,GMO5938W,LHR,PEK,17:11:57,LHR-PEK-17:11:57,24
7,HUR0974O,DEN,PVG,17:15:25,DEN-PVG-17:15:25,7
8,HZT2506M,IAH,AMS,17:12:04,IAH-AMS-17:12:04,14
9,JVY9791G,PVG,FCO,17:16:01,PVG-FCO-17:16:01,20


In [70]:
# Remove duplicate routes with lowest number of passenger ids
flights_merge=flights_merge.sort_values(by=['Route','Passenger_id']
                             , ascending=[True,False]
                             , na_position='first')
flights_merge=flights_merge[flights_merge['Route'].duplicated()== False]
# remove column of counts
flights_merge=flights_merge.drop(labels='Passenger_id', axis=1)
# flights_merge = flights_merge[flights_merge.columns[:-1]]
flights_merge

In [ ]:
# #add route info to flight only data
# route(flight_only_data)

## add to passenger data

In [73]:
#Add route info to pass_data
route(pass_data)

In [207]:
#Select data required
pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
flight_id_route = flights_merge[['Flight_id','Route']]

In [240]:
# flights_merge = 
results = []
for ind in pass_data_slim['Flight_id'].index:
    #if the flight id is correct...
    if (pass_data['Flight_id'][ind] in flight_id_route['Flight_id'].to_list()) == True:
        # take the data from passenger data
        results.append([pass_data_slim['Passenger_id'][ind]
                        ,pass_data_slim['Flight_id'][ind]
                        ,pass_data_slim['Route'][ind]])
    #if the flight id is not correct...
    else:
        true_route = (pass_data['Route'][ind])
        #  look up the route in the correct table
        row = flight_id_route[flight_id_route['Route'] == true_route].index[0]
        # give me the passenger id, the route id and the correct flight id
        results.append([pass_data_slim['Passenger_id'][ind]
                        , flight_id_route['Flight_id'][row]
                        , flight_id_route['Route'][row]])
results= pd.DataFrame(results)
    

In [241]:
results
# flights_merge

,0,1,2
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20
1,UES9151GS5,XXQ4064B,JFK-FRA-17:05:17
2,EZC9678QI6,SOH3431A,ORD-MIA-17:00:49
3,ONL0812DH1,SOH3431A,ORD-MIA-17:00:49
4,CYJ0225CH1,PME8178S,DEN-PEK-17:13:29
...,...,...,...
495,BWI0520BG6,BER7172M,KUL-LAS-17:26:07
496,LLZ3798PE3,EWH6301Y,CAN-DFW-17:22:47
497,KKP5277HZ7,KJR6646J,IAH-BKK-17:26:43
498,JJM4724RF7,XXQ4064B,JFK-FRA-17:05:17


In [111]:
results = []
for ind in pass_data_slim['Flight_id'].index:
    results.append([pass_data_slim['Flight_id'][ind],pass_data_slim['Passenger_id'][ind],pass_data_slim['Route'][ind]])
results= pd.DataFrame(results)
#     print(pass_data_slim['Flight_id'][ind])
    #     results =(pass_data_slim['Flight_id'][ind],pass_data_slim['Passenger_id'][ind],pass_data_slim['Route'][ind])
#     results=results.append(results)
#     print(pass_data_slim['Flight_id'][ind],pass_data_slim['Passenger_id'][ind],pass_data_slim['Route'][ind])
# if pass_data_slim['Flight_id'] in flights_merge['Flight_id'].to_list:
#     print(pass_data_slim['Flight_id'])

In [112]:
results

,0,1,2
0,SQU6245R,UES9151GS5,DEN-FRA-17:14:20
1,XXQ4064B,UES9151GS5,JFK-FRA-17:05:17
2,SOH3431A,EZC9678QI6,ORD-MIA-17:00:49
3,SOH3431A,ONL0812DH1,ORD-MIA-17:00:49
4,PME8178S,CYJ0225CH1,DEN-PEK-17:13:29
...,...,...,...
495,BER7172M,BWI0520BG6,KUL-LAS-17:26:07
496,EWH6301Y,LLZ3798PE3,CAN-DFW-17:22:47
497,KJR6646J,KKP5277HZ7,IAH-BKK-17:26:43
498,XXQ4064B,JJM4724RF7,JFK-FRA-17:05:17


In [ ]:
for ind in df.index: 
     print(df['Name'][ind], df['Stream'][ind]) 

In [ ]:
#Merge correct flight id
flight_id_route.columns = ['Route','correct_Flight_id']
pass_data_route = pass_data_slim.merge(flight_id_route, how='outer', on=['Route'])
pass_data_route.loc[pass_data_route['correct_Flight_id'].notna(), 'Final_Flight_id'] = pass_data_route['correct_Flight_id']
pass_data_route.loc[pass_data_route['correct_Flight_id'].isnull(), 'Final_Flight_id'] = pass_data_route['Flight_id']

In [ ]:
#Merge correct route info
flight_id_route.columns = ['Correct_Route','Flight_id']
pass_data_route = pass_data_route.merge(flight_id_route, how='outer', on=['Flight_id'])
pass_data_route.loc[pass_data_route['Correct_Route'].notna(), 'Final_Route'] = pass_data_route['Correct_Route']
pass_data_route.loc[pass_data_route['Correct_Route'].isnull(), 'Final_Route'] = pass_data_route['Route']

In [ ]:
#Select only correct data and rename
pass_data_route = pass_data_route[['Passenger_id','Final_Flight_id','Final_Route']]
pass_data_route.columns = ['Passenger_id','Flight_id','Route']

## Error Correction
### Passenger

In [ ]:
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
pass_data_route_test=pass_data_route
pass_data_route_test['Pass_id_format'] =pass_data_route['Passenger_id'].str.contains(format_pass_id)
pass_data_route_test

In [ ]:
#add column to count
pass_data_route_test['count'] = '1'

In [ ]:
#select subset of data 
pass_data_route_test = pass_data_route_test[['Passenger_id'
                      ,'Pass_id_format'
                      ,'count'
                        ]]

In [ ]:
#Group data to see passengers and total flights by passenger id
pass_only=pass_data_route_test.groupby(['Passenger_id'
                        ,'Pass_id_format'
                         ]).count()
#reset index
pass_only.reset_index(inplace=True)
pass_only

In [ ]:
#select only those whose regex is correct 
## this is not necessary as the second code will remove these, 
## however it is left here as this will be a better way to check if correct passenger id
pass_info_true = pass_only.loc[pass_only['Pass_id_format'] == True]

In [ ]:
#select only those who have more than 1 flight
pass_info_true = pass_only.loc[pass_only['count'] != 1]
#pass_info_true

In [ ]:
pass_info_false = pass_only.loc[pass_only['count'] == 1]
#pass_info_false

In [ ]:
#Create list of false passenger ids
false = pass_info_false['Passenger_id']
false

In [ ]:
#Create list of true passenger ids
true = pass_info_true['Passenger_id']
#true

In [ ]:
#create function to find the best match for a given false value and list for true values
def best_match(f,t):
    global test_best_true
    global final_true
    test_best_true = []
    final_true =[]
    for t in true:
        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    final_true=pd.DataFrame(final_true,columns=['false','true'])
   # print(final_true)

In [ ]:
# best_match(f,true)

In [ ]:
#test each false value
best_match_results = pd.DataFrame()
for f in false:
    best_match(f,true)
    best_match_results=best_match_results.append(final_true)

In [ ]:
# #create function to find the best match for a given false value and list for true values
# def best_match_table(false,true):
#     global best_match_results
#     best_match_results = pd.DataFrame()
#     for f in false:
#         global test_best_true
#         global final_true
#         test_best_true = []
#         final_true =[]
#         for t in true:
#             test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
#         test_best_true = pd.DataFrame(test_best_true)
#         max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
#         final_true.append([f,max_true])
#         final_true=pd.DataFrame(final_true,columns=['false','true'])
#        # print(final_true)
#     best_match_results=best_match_results.append(final_true)

In [ ]:
# best_match_table(false,true)

In [ ]:
best_match_results

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [ ]:
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list
break_route(pass_data_route)
from_list = pass_data_route['From_airport_IATA']

In [ ]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in from_list]
mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
reducer=mapper.groupby(['Airport_IATA_code']).sum()
reducer
reducer.reset_index(inplace=True)

In [ ]:
# merge full list of airports with number of flights from each airport
airport_list=pd.DataFrame(airport_list)
num_flights = airport_list.merge(reducer, how='outer', on=['Airport_IATA_code'])

In [ ]:
# clean up results
num_flights=num_flights.fillna(0)
num_flights=num_flights.sort_values(by=['value','Airport_IATA_code'], ascending=[False,True])
num_flights.set_index('Airport_IATA_code', inplace=True, drop=True)
num_flights

## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
# pass_data_route
# calc_arrive_time(pass_data_route)
#num_flights = airport_list.merge(from_counts_df, how='outer', on=['Airport_IATA_code'])

In [ ]:
flights = pass_data_route['Flight_id']

In [ ]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in flights]
mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
num_passengers=mapper.groupby(['Flight_id']).sum()
#num_passengers
#reducer
num_passengers.reset_index(inplace=True)

In [ ]:
# clean up results
#num_passengers=num_flights.fillna(0)
num_passengers=num_passengers.sort_values(by=['value','Flight_id'], ascending=[False,True])
num_passengers.set_index('Flight_id', inplace=True, drop=True)
num_passengers

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [ ]:
#flight info
flight_from = flight_only_deduped[['Flight_id','From_airport_IATA']]
flight_dest = flight_only_deduped[['Flight_id','Dest_airport_IATA']]

In [ ]:
#rename columns to allow merge
flight_from.columns = ['Flight_id'
                        ,'Airport_IATA_code']
flight_dest.columns = ['Flight_id'
                        ,'Airport_IATA_code']

In [ ]:
#merge from flight with latitude and longitude
from_lat_long = flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
from_lat_long.columns = ['Flight_id'
                        ,'From_Airport_IATA_code'
                        ,'From_Airport_Name'
                        ,'From_Lat'
                        ,'From_Lon'
                    ]

In [ ]:
#merge dest flight with latitude and longitude
dest_lat_lon = flight_dest.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
dest_lat_lon.columns = ['Flight_id'
                        ,'Dest_Airport_IATA_code'
                        ,'Dest_Airport_Name'
                        ,'Dest_Lat'
                        ,'Dest_Lon'
                    ]

In [ ]:
#merge from flight with latitude and longitude
from_dest_lat_lon = from_lat_long.merge(dest_lat_lon, how='outer', on=['Flight_id','Flight_id'])

In [ ]:
from_dest_lat_lon

In [ ]:
# Calculate nautical miles (a^2+b^2=c^2)
from_dest_lat_lon['Miles'] = np.sqrt((from_dest_lat_lon['Dest_Lat']-from_dest_lat_lon['From_Lat'])**2
+(from_dest_lat_lon['Dest_Lon']-from_dest_lat_lon['From_Lon'])**2)

In [ ]:
# #export results
# from_dest_lat_lon.to_csv(r'from_dest_lat_lon.csv')

In [ ]:
# Number of miles per flight 
miles_flight = from_dest_lat_lon[['Flight_id','Miles']]
miles_flight

In [ ]:
#Add flight miles to passenger data
obj4_data = pass_data_route.merge(miles_flight, how='outer', on=['Flight_id'])
obj4_data

In [ ]:
pass_mil_flight = obj4_data[['Passenger_id','Miles']]

In [ ]:
pass_mil_flight

In [ ]:
key2val2 = pd.DataFrame()
for index,row in pass_mil_flight.iterrows():
    key2val2=key2val2.append(pass_mil_flight.loc[index])
    mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
#mapper

In [ ]:
#reducer - takes key and val from mapper to provide final key and value
miles_per_pass=mapper.groupby(['Passenger_id']).sum().round(1)
#reducer
miles_per_pass.reset_index(inplace=True)
# miles_per_pass

In [ ]:
# clean up results
#num_passengers=num_flights.fillna(0)
miles_per_pass=miles_per_pass.sort_values(by=['Miles'], ascending=[False])
miles_per_pass.set_index('Passenger_id', inplace=True, drop=True)
miles_per_pass

# old code, maybe useful

In [ ]:
pass_data_df.value_counts(['From_airport_IATA'])

In [ ]:
pass_data_df.groupby(['From_airport_IATA']).count()

In [ ]:
flight_only_grp.isin(flight_only_grp['From_airport_IATA'])

In [ ]:
#pass_data_df.set_index(["From_airport_IATA"]).count(level="From_airport_IATA")
pass_data_df.count(axis="From_airport_IATA")


In [ ]:
#pd.concat([obj_1_list_df, from_counts_df], keys=['Airport_IATA_code','Airport_IATA_code'])


In [ ]:
#pass_data_df['Depart_time_epoch'].dt.strftime("%m/%d/%y")
#pass_data_df['Depart_time_epoch']=
(pd.to_datetime(pass_data_df['Depart_time_epoch'],unit='s')) 


In [ ]:
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [ ]:
if re.match(r"hello[0-9]+", 'hello1'):
    print('Yes')

In [ ]:
pattern = r"[A-Z][A-Z][A-Z]"

pd.Series(
     ["DEN", "A;S", "AM%", "3b", "03c", "4dx"],
 ).str.contains(pattern)

In [ ]:
#pass_data_df[pass_data_df['From_airport_IATA'].str.match('DEN')== True]
#pass_data_df['From_airport_format']=
pass_data_df['From_airport_IATA'].str.match('[A-Z][A-Z][A-Z]')

In [ ]:
#merging by IATA code
#select remaining from airports
from_values = flight_info_true.From_airport_IATA.unique()
dest_values = flight_info_true.Dest_airport_IATA.unique()
#turn to dataframe
from_values_df = pd.DataFrame(from_values)
dest_values_df = pd.DataFrame(dest_values)
#rename columns to allow merge
from_values_df.columns = ['Airport_IATA_code']
dest_values_df.columns = ['Airport_IATA_code']

In [ ]:
from_merge['From_airport_exists']=from_merge['Airport_name'].str.contains(r"[A-Z]")
dest_merge['Dest_airport_exists']=dest_merge['Airport_name'].str.contains(r"[A-Z]")
#from_merge[from_merge['Airport_name'].str.contains('NaN')==False]
#new_df = df[df["col"].str.contains(word) == False]

In [ ]:
import Levenshtein as lev


In [ ]:
import pandas as pd
from textdistance import levenshtein

attempts = [['passw0rd', 'pasw0rd'],
            ['passwrd', 'psword'],
            ['psw0rd', 'passwor'],
            ['helloworld', 'heloworl']]

df=pd.DataFrame(attempts, columns=['password', 'attempt'])

df.apply(lambda x: levenshtein.distance(x['password'],  x['attempt']), axis=1)

In [ ]:
#best_match_results_pd[['results','true']] = 
best_match_results_pd['results'].str.split('-',expand=True)
#df[['V','allele']] = df['V'].str.split('-',expand=True)


In [ ]:
best_match_results_pd.iloc[1, 0]

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similar("Apple","Appel")

In [ ]:
round(SequenceMatcher(None, 'WBE6935NU', 'HCA3158QA6').ratio(),3)
round(SequenceMatcher(None, 'UES9151GS5', 'UES915*GS5').ratio(),3)
round(SequenceMatcher(None, 'UES915*GS5', 'UES9151GS5').ratio(),3)


In [ ]:
install python-Levenshtein

In [ ]:
f = 'SPRb484HA6'
test_best_true = pd.DataFrame()
for t in true:
    ratio = round(SequenceMatcher(None, t, f).ratio(),3)
    print(t, ratio)
    #test_best_true = test_best_true.append(ratio)


In [ ]:
#final_true
#str(final_true)[1:-1] 
#test_best_true
#max_true

In [ ]:
# #Calculate epoch Arrival Time
# pass_data['Arrive_epoch'] = pass_data['Depart_time_epoch']+pass_data['Flight_time_min']*60
# #Convert epoch to datetime
# pass_data['Depart_time_datetime'] = pd.to_datetime(pass_data['Depart_time_epoch'],unit='s')
# pass_data['Arrive_time_datetime'] = pd.to_datetime(pass_data['Arrive_epoch'],unit='s')

In [ ]:
# #And new datetime formats
# pass_data['Depart_day_month']=pass_data['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
# #pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
# pass_data['Depart_time']=pass_data['Depart_time_datetime'].dt.strftime("%H:%M:%S")
# pass_data['Arrive_time']=pass_data['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [ ]:
# route_info=pass_data[['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time'
#                      ]]

# route_info=route_info.groupby(['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time'          
#                          ]).count()
# route_info.reset_index(inplace=True)


In [ ]:
# #select subset of data 
# flight_only_data = pass_data[['Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'         
#                       ,'Flight_time_min'
#                       ,'count'
#                         ]]
# 
# #Group data by flight
# flight_only_data = flight_only_data.groupby(['Flight_id'
#                           ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'         
#                       ,'Flight_time_min'           
#                          ]).count()
# #reset index
# flight_only_data.reset_index(inplace=True)

# #checks
# flight_only_data
# len(flight_only_data)


In [ ]:
# #select subset of data 
# flight_only_data = merge_with_dest[['Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_time'
#                       ,'count'
#                         ]]
# #Group data to see total flights
# flight_only_data = flight_only_data.groupby(['Flight_id'
#                         ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_time'
#                          ]).sum()
# flight_only_data

In [ ]:
#send to csv for checking
#merge_with_dest.to_csv(r'merge_with_dest.csv')

In [ ]:
#pass_data_route.to_csv((r'pass_data_route.csv'))
# pass_data_route

In [ ]:
#flight_only_data['Route'] = flight_only_data['From_airport_IATA']+'-' + flight_only_data['Dest_airport_IATA']+'-' + flight_only_data['Depart_time']

In [ ]:
#Add column to allow counting of passengers
# pass_data['count']='1'

#### For debugging only

In [ ]:
# pass_data = pd.read_csv('AComp_Passenger_data_no_error.csv',header=None)   #comment out as necessary
# pass_datet = pd.read_csv('AComp_Passenger_data_no_error_DateTime.csv',header=None)
# pass_datetime.columns = ['Passenger_id'
#                         ,'Flight_id'
#                         ,'From_airport_IATA'
#                         ,'Dest_airport_IATA'
#                         ,'Departure_date_time'
#                         ,'Depart_time_epoch'
#                         ,'Flight_time_min'
#                         ,'Arrive_date_time'
# ]

#### Map Reduce testing

In [ ]:
#map flights to key value pair [airport,1]
key2val2 = []
for a in from_list:
    key2val2.append([a,1])

key2val2
df=pd.DataFrame(key2val2,columns=['key','value'])
df
#reduce
#sum(key2val2)

In [ ]:
#map flights to key value pair [airport,1]
val2sum =[]
for a in from_list:
    val2 =1
    val2sum.append(val2)
#reduce
sum(val2sum)

#### Obj 1 old method

In [ ]:
# List of top 30 airports
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list.columns = ['From_airport_IATA']
airport_list = pd.DataFrame(airport_list)

In [ ]:
# Add all route info
pass_data_route=pass_data_route.merge(route_info, how='inner', on=['Route'])

In [ ]:
# Count number of departure flights
from_df = pass_data_route['From_airport_IATA']
from_counts = from_df.value_counts()
from_counts_df = pd.DataFrame(from_counts)
# move index to column
from_counts_df['Airport_IATA_code'] =from_counts_df.index

In [ ]:
#Check data
print(from_df)
print(from_counts)
print(from_counts_df)

#### trying to do mapreduce obj 4

In [ ]:
# #mapper, where key = airport and value = 1 for each airport in from_list
# key2val2 = [[a, 1] for a in flights]
# mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
# #mapper

# #reducer - takes key and val from mapper to provide final key and value
# num_passengers=mapper.groupby(['Flight_id']).sum()
# #num_passengers
# #reducer
# num_passengers.reset_index(inplace=True)

In [ ]:
key2val2 = pd.DataFrame()
key2val2.append(pass_mil_flight, ignore_index=True)

# for index,row in pass_mil_flight.iterrows():
#     key2val2.append(pd.Series(['Passenger_id','Miles'], index=pass_mil_flight.columns ), ignore_index=True)
#     print(row['Passenger_id'],row['Miles'])
#     df=pd.DataFrame(key2val2,columns=['Passenger_id','Miles'])

    
# >>> df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
# >>> df
#    A  B
# 0  1  2
# 1  3  4
# >>> df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
# >>> df.append(df2)
#    A  B
# 0  1  2
# 1  3  4
# 0  5  6
# 1  7  8    
    
    
    
#     modDfObj = dfObj.append(pd.Series(['Raju', 21, 'Bangalore', 'India'], index=dfObj.columns ), ignore_index=True)
#modDfObj = dfObj.append({'Name' : 'Sahil' , 'Age' : 22} , ignore_index=True)

    #map flights to key value pair [airport,1]
# key2val2 = []
# for a in from_list:
#     key2val2.append([a,1])

# key2val2
# df=pd.DataFrame(key2val2,columns=['key','value'])
# df
#reduce
#sum(key2val2)
# key2val2
# df=pd.DataFrame(key2val2,columns=['Passenger_id','Miles'])
# df
#reduce
#sum(key2val2)

# import pandas as pd

# df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})



In [ ]:
# import pandas as pd

# df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})

# for index, row in df.iterrows():
#     print(row['c1'], row['c2'])

In [ ]:
#mapper, where key = airport and value = 1 for each airport in from_list
# key2val2 = [[p, m] for p in pass_mil_flight]
# mapper = pd.DataFrame(key2val2, columns=['Passenger_id','Miles'])
# #mapper

# #reducer - takes key and val from mapper to provide final key and value
# num_passengers=mapper.groupby(['Flight_id']).sum()
# #num_passengers
# #reducer
# num_passengers.reset_index(inplace=True)

#### Obj 4 old way

In [ ]:
#Number of miles per flight per passenger
pass_flights= pd.pivot_table(obj4_data, values='Miles', index=['Passenger_id'],
               columns=['Flight_id']
              , aggfunc=np.sum
              )
pass_flights

In [ ]:
#pass_flights.to_csv(r'pass_flights.csv')

In [ ]:
#Add up flight miles per passenger
Pass_miles = pass_flights.sum(axis = 1, skipna = True) 
Pass_miles_df = pd.DataFrame(Pass_miles)
Pass_miles_df.columns = ['Air_miles']
Pass_miles_df = Pass_miles_df.sort_values(by=['Air_miles'], ascending=False)
Pass_miles_df.to_csv(r'Pass_miles_df.csv')
Pass_miles_df

#### Trying to do passenger error correction again!

In [ ]:
#create function to find the best match for a given false value and list for true values
#def best_match(f,t):
 #   global test_best_true
  #  global final_true
for f in false:
    test_best_true = []
    final_true =[]
    for t in true:
           test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    final_true=pd.DataFrame(final_true,columns=['false','true'])
    #final_true = pd.DataFrame (final_true)
   # print(final_true)

# key2val2 = [[a, 1] for a in from_list]
# mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])

# #map flights to key value pair [airport,1]
# key2val2 = []
# for a in from_list:
#     key2val2.append([a,1])

# key2val2
# df=pd.DataFrame(key2val2,columns=['key','value'])
# df
#reduce
#sum(key2val2)

In [ ]:
final_true
key2val2 = pd.DataFrame()
for index,row in pass_mil_flight.iterrows():
    key2val2=key2val2.append(pass_mil_flight.loc[index])
    mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
#mapper

In [ ]:
for f in false:
#def best_match(list):
    test_best_true = []
    final_true =[]
    for t in true:
           test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    final_true=pd.DataFrame(final_true,columns=['false','true'])


In [ ]:
# final_true

In [ ]:
# final_true
# key2val2 = pd.DataFrame()
# for index,row in pass_mil_flight.iterrows():
#     key2val2=key2val2.append(pass_mil_flight.loc[index])
#     mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
# #mapper

In [ ]:
# #turn into dataframe
# best_match_results_pd=pd.DataFrame (best_match_results)
# best_match_results_pd.columns = ['results']
# best_match_results_pd
# #obj_1_list.columns = ['From_airport_IATA']

In [ ]:
#pass_data_route['From_airport_IATA']
#best_match_results_pd['results'].str.split('-',expand=True)
#route(pass_data_route)
pass_data_route
route_info

#### Previous obj 2 

In [ ]:
# #obj2_results = 
# pass_data_route.groupby(['Route'
#                       ,'Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Arrive_time'
#                       ,'Flight_time_min']).count()
# #reset index as columns
# #obj2_results.reset_index(inplace=True)
# #obj2_results

#### Old Flight error correction

In [ ]:
#correct formats
format_IATA = r"[A-Z][A-Z][A-Z]"
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"

#format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
#flight_only_data['Pass_id_format'] =flight_only_data['Passenger_id'].str.contains(format_pass_id)

#add column indicating if correct
flight_only_data['From_airport_format'] =flight_only_data['From_airport_IATA'].str.contains(format_IATA)
flight_only_data['Dest_airport_format'] =flight_only_data['Dest_airport_IATA'].str.contains(format_IATA)
flight_only_data['Flight_id_format'] =flight_only_data['Flight_id'].str.contains(format_flight_id)

#add column indicating if all flight regex correct
flight_only_data['Flight_info_correct']=flight_only_data.all(axis='columns')

#select only those whose regex is correct
flight_info_true = flight_only_data.loc[flight_only_data['Flight_info_correct'] == True]


In [ ]:
#Take True airport values, change column names to enable merge

true_airports_from = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_from.columns = ['From_airport_IATA']
#add column to know if exists
true_airports_from['From_airport_exists']=True

true_airports_dest = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_dest.columns = ['Dest_airport_IATA']
#add column to know if exists
true_airports_dest['Dest_airport_exists']=True

#merge with from
merge_with_from=flight_info_true.merge(true_airports_from
                     , how='outer', on=['From_airport_IATA'])
#merge with dest
merge_with_dest=merge_with_from.merge(true_airports_dest
                     , how='outer', on=['Dest_airport_IATA'])
#check data
#merge_with_dest

#drop NaNs - any that are not in the true airport list
merge_with_dest=merge_with_dest.dropna(subset=['From_airport_exists', 'Dest_airport_exists'])


In [ ]:

#select only flight info 
flight_only_data = merge_with_dest[flight_info]
#add a column to allow count
#add_1(flight_only_data)
#Group data by flight
flight_only_data = flight_only_data.groupby(flight_info).sum()
#reset index
flight_only_data.reset_index(inplace=True)

In [ ]:
#sort, route in alpha-numeric order, count of passenger id in descending (lowest value last)
flight_only_data=flight_only_data.sort_values(by=['Route','Passenger_id']
                             , ascending=[True,False]
                             , na_position='first')

In [ ]:
# Remove duplicate with lowest number of passenger ids
flight_only_data['Route_dupe']=flight_only_data['Route'].duplicated()
flight_only_deduped = flight_only_data[flight_only_data.Route_dupe == False]

In [ ]:
len(flight_only_deduped)